In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.spatial import distance
import tensorflow as tf
from sklearn.model_selection import train_test_split
import dama as dm
import pickle
import time
import gc
import sys
import awkward as ak

from freedom.toy_model.toy_model_functions import toy_model
from freedom.toy_model.detectors import get_spherical_detector
from types import SimpleNamespace
from freedom.toy_model import NNs

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16 
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 14

par_names = ['x', 'y', 'z', 't', 'azi', 'zen', 'E', 'trck_frac']

def plot_truth(axes, truth, idx=(0,1)):
    if not isinstance(axes, np.ndarray):
        axes = np.array([axes])
    for ax in axes.flatten():
        ax.plot([truth[idx[0]]], [truth[idx[1]]], marker='$T$', markersize=10, color='k')

def plot_diff(a, b, axes, title_a='a', title_b='b', vmax=None, limit_diff=False, **kwargs):
    
    levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
    labels = [str(i) + r'$\sigma$' for i in range(1,6)]
    colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
    colors = plt.cm.viridis(np.linspace(0, 0.9, 6))
    #a.plot(ax=axes[0], cmap='Greys', label=r'$\Delta LLH$', **kwargs)
    a.plot_contour(ax=axes[0], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[0].set_title(title_a)
    #b.plot(ax=axes[1], cmap='Greys', label=r'$\Delta LLH$', **kwargs)
    b.plot_contour(ax=axes[1], levels=levels,  labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[1].set_title(title_b)
    diff = a - b
    if limit_diff:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-vmax, vmax=vmax, label=r'$\Delta LLH$', **kwargs)
        #diff.plot_contour(ax=axes[2], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
        #np.clip(-diff, 0, None).plot_contour(ax=axes[2], levels=[0.1,0.2, 0.3], colors=['r']*2)
    else:
        diff.plot(ax=axes[2], cmap='RdBu', cbar=True, vmin=-np.max(np.abs(diff)), vmax=np.max(np.abs(diff)), label=r'$\Delta LLH$', **kwargs) 
        #diff.plot_contour(ax=axes[2], levels=levels, labels=labels, colors=colors, label=r'$\Delta LLH$', **kwargs)
    axes[2].set_title(title_a + ' - ' + title_b)
    
def correct_azi(azi):
    azi = np.where(azi<-np.pi, azi+2*np.pi, azi)
    return np.where(azi>np.pi, azi-2*np.pi, azi)

In [ ]:
detector = get_spherical_detector(radius=10, subdivisions=4)
print(len(detector))

fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(projection='3d')
ax.scatter(detector[:,0], detector[:,1], detector[:,2], color='black', s=50)
ax.grid(False)
#ax.scatter(3., 1., 0.)
#plt.savefig('images/spherical_det.png', bbox_inches='tight')

In [ ]:
toy_experiment = toy_model(detector)

In [ ]:
truth = np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8])

In [ ]:
# generate one test event
test_event = toy_experiment.generate_event(truth)

In [ ]:
%%time
# Grid scan

g = dm.GridData(x=np.linspace(-10, 10, 100), y=np.linspace(-10, 10, 100))
IDX = (0,1)

g['dom_hit_term'] = np.empty(g.shape)
g['dom_charge_terms'] = np.empty(g.shape)
g['total_charge_hit_terms'] = np.empty(g.shape)
g['total_charge_terms'] = np.empty(g.shape)

p = np.copy(truth)

for idx in np.ndindex(g.shape):
    p[IDX[0]] = g['x'][idx]
    p[IDX[1]] = g['y'][idx]
    segments = toy_experiment.model(*p)
    g['dom_hit_term'][idx] = toy_experiment.nllh_p_term_dom(segments, test_event[0])
    g['dom_charge_terms'][idx] = toy_experiment.nllh_N_term_dom(segments, test_event[1])
    g['total_charge_hit_terms'][idx] = toy_experiment.nllh_p_term_tot(segments, test_event[0])
    g['total_charge_terms'][idx] = toy_experiment.nllh_N_term_tot(segments, test_event[1])
    
g['dom_hit_term'] -= g['dom_hit_term'].min()
g['dom_charge_terms'] -= g['dom_charge_terms'].min()
g['dom_llh'] = g['dom_hit_term'] + g['dom_charge_terms']
g['total_charge_hit_terms'] -= g['total_charge_hit_terms'].min()
g['total_charge_terms'] -= g['total_charge_terms'].min()
g['total_charge_llh'] = g['total_charge_hit_terms'] + g['total_charge_terms']
g['dom_llh'] -= g['dom_llh'].min()
g['total_charge_llh'] -= g['total_charge_llh'].min()

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

plot_diff(g['dom_hit_term'], g['total_charge_hit_terms'], axes=ax[0], title_a='per DOM hit', title_b='total hit', vmax=20, limit_diff=True)
plot_diff(g['dom_charge_terms'], g['total_charge_terms'], axes=ax[1], title_a='per DOM charge', title_b='total charge', vmax=20, limit_diff=True)
plot_diff(g['dom_llh'], g['total_charge_llh'], axes=ax[2], title_a='per DOM llh', title_b='total llh', limit_diff=False)

plot_truth(ax, truth, IDX)
#plt.savefig('images/LLH_decompose.png', bbox_inches='tight')

# Train NNs

In [ ]:
NE, Set = 100_000, 3
events, meta = toy_experiment.generate_events(n=NE, gamma=0, gen_volume="sphere", e_lim=(1,12), inelast_lim=(0,1),
                                              t_width=0, radius=10, contained=True, min_hits=3, rand=Set)
#truths = NNs.make_truth_array(events)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
nGPUs = strategy.num_replicas_in_sync

In [ ]:
#del(x), del(t)
#gc.collect()

## Hit Net - total charge

In [ ]:
x, t = NNs.get_hit_data(events)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs, time_spread=10)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs, time_spread=10)

#d_train = NNs.DataGenerator(train_test_split(ak.ravel(events.photons[['x', 'y', 'z', 't', 'q', 'sensor_id']]).to_numpy().reshape(6, -1).T, test_size=1, random_state=42)[0],
#                            train_test_split(np.repeat(NNs.make_truth_array(events), ak.count(events.photons.t, axis=1).to_numpy(), axis=0), test_size=1, random_state=42)[0], 
#                            batch_size=4096*nGPUs, time_spread=10)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)

with strategy.scope():
    hmodel_t = NNs.get_hmodel(x_shape=6, t_shape=8, trafo=NNs.hit_trafo_3D, activation='swish', final_activation='swish')
    #hmodel_t = tf.keras.models.load_model('networks/spherical/spherical_toy_hitnet_total3.h5',
    #                                      custom_objects={'hit_trafo_3D':NNs.hit_trafo_3D})
    hmodel_t.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = hmodel_t.fit(d_train, epochs=5, verbose=1, validation_data=d_valid)

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = hmodel_t.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel_t.layers[-1].activation = tf.keras.activations.linear
hmodel_t.compile()

llhs = -hmodel_t((xxs, tts)).numpy()
llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g.hit_llh_total = llhs.reshape(g.shape)
g.hit_llh_total -= g.hit_llh_total.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.total_charge_hit_terms, g.hit_llh_total, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)
#plt.savefig('images/hitNNtest.png', bbox_inches='tight')

In [ ]:
hmodel_t.save('networks/spherical/'+str(NE)+'/spherical_toy_hitnet_total_'+str(NE)+'_set'+str(Set)+'.h5')
#hmodel_t = tf.keras.models.load_model('networks/spherical/spherical_toy_hitnet_total.h5',
#                                      custom_objects={'hit_trafo_3D':NNs.hit_trafo_3D})

## Charge Net - total charge

In [ ]:
x, t = NNs.get_charge_data(events)
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs)
d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs)

#d_train = NNs.DataGenerator(train_test_split(np.stack([np.sum(events.n_obs.to_numpy(), axis=1), np.sum(events.n_obs.to_numpy()>0, axis=1)], axis=1), test_size=1, random_state=42)[0],
#                            train_test_split(NNs.make_truth_array(events), test_size=1, random_state=42)[0],
#                            batch_size=4096*nGPUs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4) #3

with strategy.scope():
    cmodel = NNs.get_cmodel(x_shape=2, t_shape=8, trafo=NNs.charge_trafo_3D, activation='swish', final_activation='swish')
    #cmodel = tf.keras.models.load_model('networks/spherical/spherical_toy_chargenet_total3.h5',
    #                                     custom_objects={'charge_trafo_3D':NNs.charge_trafo_3D})
    cmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = cmodel.fit(d_train, epochs=70, verbose=1, validation_data=d_valid) #

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = cmodel.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xxs = np.tile([len(test_event[0]), len(np.unique(test_event[0][:,0]))], np.prod(g.shape))
xxs = xxs.reshape(-1, 2)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)

cmodel.layers[-1].activation = tf.keras.activations.linear
cmodel.compile()

llhs = np.nan_to_num(-cmodel((xxs, tts)))

g.charge_llh_total = llhs.reshape(g.shape)
g.charge_llh_total -= g.charge_llh_total.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.total_charge_terms, g.charge_llh_total, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)
#plt.savefig('images/chargeNNtest.png', bbox_inches='tight')

In [ ]:
cmodel.save('networks/spherical/'+str(NE)+'/spherical_toy_chargenet_total_'+str(NE)+'_set'+str(Set)+'.h5')
#cmodel = tf.keras.models.load_model('networks/spherical/spherical_toy_chargenet_total.h5',
#                                     custom_objects={'charge_trafo_3D':NNs.charge_trafo_3D})

## LLH - total charge

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_diff(g.total_charge_hit_terms, 
          g.hit_llh_total, 
          title_a='Hit Analytic', title_b='Hit NN', vmax=10, axes=ax[0], limit_diff=True)

plot_diff(g.total_charge_terms, 
          g.charge_llh_total, 
          title_a='Charge Analytic', title_b='Charge NN', vmax=10, axes=ax[1], limit_diff=True)

ana, NN = g.total_charge_hit_terms+g.total_charge_terms, g.hit_llh_total+g.charge_llh_total
plot_diff(ana-ana.min(), 
          NN-NN.min(), 
          title_a='Analytic', title_b='NN', vmax=10, axes=ax[2], limit_diff=True)
plot_truth(ax, truth, IDX)

#plt.savefig('images/NNtest_totalC.png', bbox_inches='tight')

## Hit Net - per dom

In [ ]:
#x, t = NNs.get_hit_data(events)
#x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
#d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=10)
#d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs, shuffle='inDOM', time_spread=10)

d_train = NNs.DataGenerator(train_test_split(ak.ravel(events.photons[['x', 'y', 'z', 't', 'q', 'sensor_id']]).to_numpy().reshape(6, -1).T, test_size=1, random_state=42)[0],
                            train_test_split(np.repeat(NNs.make_truth_array(events), ak.count(events.photons.t, axis=1).to_numpy(), axis=0), test_size=1, random_state=42)[0], 
                            batch_size=4096*nGPUs, shuffle='inDOM', time_spread=10)

In [ ]:
optimizer = tf.keras.optimizers.Adam(2e-5)

with strategy.scope():
    #hmodel_d = NNs.get_hmodel(x_shape=6, t_shape=8, trafo=NNs.hit_trafo_3D, activation='swish', final_activation='swish')
    hmodel_d = tf.keras.models.load_model('networks/spherical/spherical_toy_hitnet_dom3.h5',
                                          custom_objects={'hit_trafo_3D':NNs.hit_trafo_3D})
    hmodel_d.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = hmodel_d.fit(d_train, epochs=3, verbose=1) #, validation_data=d_valid

In [ ]:
plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = hmodel_d.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xxs = np.repeat(test_event[0][np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 6)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(test_event[0]), axis=0)

hmodel_d.layers[-1].activation = tf.keras.activations.linear
hmodel_d.compile()

llhs = -hmodel_d.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(test_event[0]))), axis=1)

g.hit_llh_dom = llhs.reshape(g.shape)

g.hit_llh_dom -= g.hit_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_hit_term, g.hit_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)
#plt.savefig('images/hitNNtest.png', bbox_inches='tight')

In [ ]:
#hmodel_d.save('networks/spherical/spherical_toy_hitnet_dom3.h5')
hmodel_d = tf.keras.models.load_model('networks/spherical/spherical_toy_hitnet_dom.h5',
                                      custom_objects={'hit_trafo_3D':NNs.hit_trafo_3D})

## Charge Net - per DOM

In [ ]:
#x, t = NNs.get_dom_data(events, detector)
#x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1, random_state=42)
#d_train = NNs.DataGenerator(x_train, t_train, batch_size=4096*nGPUs)
#d_valid = NNs.DataGenerator(x_test, t_test, batch_size=4096*nGPUs)

d_train = NNs.DataGenerator(train_test_split(np.hstack([np.repeat(detector[np.newaxis, :], len(events), axis=0).reshape(-1, 3), ak.ravel(events.n_obs).to_numpy()[:, np.newaxis]]), test_size=1, random_state=42)[0],
                            train_test_split(np.repeat(NNs.make_truth_array(events), len(detector), axis=0), test_size=1, random_state=42)[0], 
                            batch_size=4096*nGPUs)

In [ ]:
optimizer = tf.keras.optimizers.Adam(5e-4)

with strategy.scope():
    #dmodel = NNs.get_hmodel(x_shape=4, t_shape=8, trafo=NNs.dom_trafo_3D, activation='swish', final_activation='swish')
    dmodel = tf.keras.models.load_model('networks/spherical/spherical_toy_chargenet_dom3.h5',
                                         custom_objects={'dom_trafo_3D':NNs.dom_trafo_3D})
    dmodel.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
hist = dmodel.fit(d_train, epochs=2, verbose=1) #, validation_data=d_valid

In [ ]:
plt.plot(hist.history['loss'])
#plt.plot(hist.history['val_loss'])
plt.yscale('log')

In [ ]:
inp, lab = d_valid.__getitem__(0)
pred = dmodel.predict(inp, batch_size=4096).flatten()
plt.hist(pred[lab==0], 100, histtype='step')
plt.hist(pred[lab==1], 100, histtype='step');
plt.yscale('log')

In [ ]:
xx = []
ind = test_event[0][:, 5]
for i in range(len(detector)):
    d = np.append(detector[i], np.sum(ind==i))
    xx.append(list(d))
xxs = np.repeat(np.array(xx)[np.newaxis, :], np.prod(g.shape), axis=0)
xxs = xxs.reshape(-1, 4)

tts = np.repeat(truth[np.newaxis, :], np.prod(g.shape), axis=0)
tts[:, IDX[0]] = g.get_array('x', flat=True)
tts[:, IDX[1]] = g.get_array('y', flat=True)
tts = np.repeat(tts, len(detector), axis=0)

dmodel.layers[-1].activation = tf.keras.activations.linear
dmodel.compile()

llhs = -dmodel.predict((xxs, tts), batch_size=4096)

llhs = np.sum(np.nan_to_num(llhs.reshape(-1, len(detector))), axis=1)

g.charge_llh_dom = llhs.reshape(g.shape)

g.charge_llh_dom -= g.charge_llh_dom.min()

fig, ax = plt.subplots(1, 3, figsize=(20,5))
plt.subplots_adjust(wspace=0.3)

plot_diff(g.dom_charge_terms, g.charge_llh_dom, title_a='Analytic', title_b='NN', vmax=10, axes=ax, limit_diff=True)
plot_truth(ax, truth, IDX)
#plt.savefig('images/hitNNtest.png', bbox_inches='tight')

In [ ]:
#dmodel.save('networks/spherical/spherical_toy_chargenet_dom3.h5')
dmodel = tf.keras.models.load_model('networks/spherical/spherical_toy_chargenet_dom.h5',
                                     custom_objects={'dom_trafo_3D':NNs.dom_trafo_3D})

## LLH - per DOM

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(20,17))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_diff(g.dom_hit_term, 
          g.hit_llh_dom, 
          title_a='Hit Analytic', title_b='Hit NN', vmax=10, axes=ax[0], limit_diff=True)

plot_diff(g.dom_charge_terms, 
          g.charge_llh_dom, 
          title_a='Charge Analytic', title_b='Charge NN', vmax=10, axes=ax[1], limit_diff=True)

ana, NN = g.dom_hit_term+g.dom_charge_terms, g.hit_llh_dom+g.charge_llh_dom
plot_diff(ana-ana.min(), 
          NN-NN.min(), 
          title_a='Analytic', title_b='NN', vmax=10, axes=ax[2], limit_diff=True)
plot_truth(ax, truth, IDX)

#plt.savefig('images/NNtest_perDOM.png', bbox_inches='tight')

## LLH - both

In [ ]:
def plot_overlay(a, b, ax, **kwargs):
    levels = stats.chi2(df=2).isf(stats.norm.sf(np.arange(1,6))*2)/2    
    labels = [str(i) + r'$\sigma$' for i in range(1,6)]
    colors = plt.cm.viridis(np.linspace(0, 0.9, 6))
    a.plot_contour(ax=ax, levels=levels, labels=labels, colors=colors, **kwargs)
    b.plot_contour(ax=ax, levels=levels, linestyles=[':']*len(levels), colors=colors, **kwargs)
    ax.plot([], [], label='Analytic', color='Tab:blue')
    ax.plot([], [], label='NN', linestyle=':', color='Tab:blue')
    ax.set_xlabel(par_names[IDX[0]])
    ax.set_ylabel(par_names[IDX[1]])

In [ ]:
g['llh_dom'] = g['hit_llh_dom'] + g['charge_llh_dom']
g['llh_total'] = g['hit_llh_total'] + g['charge_llh_total']
g['llh_dom'] -= g['llh_dom'].min()
g['llh_total'] -= g['llh_total'].min()

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20,12))
plt.subplots_adjust(wspace=0.25, hspace=0.25)

plot_truth(ax, truth, IDX)

ax[0][0].set_title('HitNet', size=20)
plot_overlay(g.dom_hit_term, g.hit_llh_dom, ax[0][0])
ax[0][0].text(g.x.min()-2.5, g.y.mean(), 'Per Sensor', rotation=90, size=20, ha='center', va='center')

ax[0][1].set_title('ChargeNet', size=20)
plot_overlay(g.dom_charge_terms, g.charge_llh_dom, ax[0][1])

ax[0][2].set_title('Complete LLH', size=20)
plot_overlay(g.dom_llh, g.llh_dom, ax[0][2])
ax[0][2].legend(loc='upper left')

plot_overlay(g.total_charge_hit_terms, g.hit_llh_total, ax[1][0])
ax[1][0].text(g.x.min()-2.5, g.y.mean(), 'Total Detector', rotation=90, size=20, ha='center', va='center')

plot_overlay(g.total_charge_terms, g.charge_llh_total, ax[1][1])

plot_overlay(g.total_charge_llh, g.llh_total, ax[1][2])

#plt.savefig('images/NNtest_spherical_xt.png', bbox_inches='tight')

# Event reweighting

In [ ]:
def get_hit_data(truth, vec, bins):
    t = truth.reshape((1,8))
    ts = np.repeat(t, len(hits), axis=0)

    r_t = np.exp(hmodel_t.predict([hits, ts], batch_size=5000)).flatten()
    r_d = np.exp(hmodel_d.predict([hits, ts], batch_size=5000)).flatten()

    dists = distance.cdist(detector[:,:3], toy_experiment.model(*truth)[:,:3])
    survive = toy_experiment.survival(dists)
    hit_llh_p, hit_llh_ps = [], []
    for b in bins:
        mat = toy_experiment.pandel.pdf(b-dists*4.333, d=dists)
        hit_llh_p.append(np.sum(np.sum(mat, axis=0) * vec))
        hit_llh_ps.append(np.sum(np.sum(mat * survive, axis=0) * vec))
    norm_p = np.sum(np.array(hit_llh_p)) * np.diff(bins)[0]
    norm_ps = np.sum(np.array(hit_llh_ps)) * np.diff(bins)[0]
    
    return r_t, r_d, hit_llh_p, hit_llh_ps, norm_p, norm_ps

def get_charge_data(truth, exp_bins, exp_bins_fine):
    t = truth.reshape((1,8))
    ts_t = np.repeat(t, len(charges_t), axis=0)
    ts_d = np.repeat(t, len(charges_d), axis=0)

    r_t = np.exp(cmodel.predict([charges_t, ts_t], batch_size=5000)).flatten()
    r_d = np.exp(dmodel.predict([charges_d, ts_d], batch_size=5000)).flatten()

    N_exp = toy_experiment.N_exp(toy_experiment.model(*truth))
    dom_c_llh = np.zeros(len(exp_bins))
    for N in N_exp:
        dom_c_llh += stats.poisson.pmf(exp_bins, mu=N)
        
    return r_t, r_d, N_exp, dom_c_llh

In [ ]:
events, meta = toy_experiment.generate_events(n=100_000, gamma=0, gen_volume="sphere", e_lim=(1,12), inelast_lim=(0,1),
                                              t_width=0, radius=10, contained=True, min_hits=3)
truths = NNs.make_truth_array(events)

In [ ]:
#test_events = []
#while len(test_events) < 10_000:
#    e = toy_experiment.generate_event(truth)
#    if len(e[0]) >= 3:
#        test_events.append(e)

hitnets

In [ ]:
#%%time
#for i, e in enumerate(test_events):
#    if i == 0:
#        test_hits = e[0]
#    else:
#        test_hits = np.concatenate([test_hits, e[0]])

In [ ]:
%%time
hits = NNs.get_hit_data(events)[0]
hits[:, 3] += np.random.normal(0, 10, len(hits))

In [ ]:
bins = np.linspace(-20,200,100)

r_t, r_d, hit_llh_p, hit_llh_ps, norm_p, norm_ps = get_hit_data(
                                                            np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8]), 
                                                            [4,0.2,0.2,0.2,0.2,0.2],
                                                            bins)

r_t2, r_d2, hit_llh_p2, hit_llh_ps2, norm_p2, norm_ps2 = get_hit_data(
                                                            np.array([-5., 7., 0, 0, 0, np.arccos(1), 3., 1]), 
                                                            [3],
                                                            bins)

r_t3, r_d3, hit_llh_p3, hit_llh_ps3, norm_p3, norm_ps3 = get_hit_data(
                                                            np.array([-5., 3., 0, 0, 0, np.arccos(1), 4., 0.9]), 
                                                            [3.6,0.2,0.2],
                                                            bins)

In [ ]:
plt.figure(figsize=(14,7*0.618))

plt.subplot(121)
#plt.hist(test_hits[:, 3], bins, label='Pulses example event', density=True, histtype='step')
plt.hist(hits[:, 3], bins, label=r'$p(x)$', density=True, histtype='step', color='black')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps)/norm_ps, c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t2, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps2)/norm_ps2, c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_t3, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_ps3)/norm_ps3, c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.legend()
plt.title('All-sensor')
plt.xlabel(r'$x$ = Hit time')
plt.ylabel('PDF')


plt.subplot(122)
#plt.hist(test_hits[:, 3], bins, label='Pulses example event', density=True, histtype='step')
plt.hist(hits[:, 3], bins, label=r'$p(x)$', density=True, histtype='step', color='black')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p)/norm_p, c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p2)/norm_p2, c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3, density=True, histtype='step')
plt.plot(bins+np.diff(bins)[0]/2, np.array(hit_llh_p3)/norm_p3, c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.legend()
plt.title('Per-sensor')
plt.xlabel(r'$x$ = Hit time')

#plt.savefig('images/spherical/NNtest_spherical_rweight_time.pdf', bbox_inches='tight')

In [ ]:
h = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
h2 = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t2, density=True, histtype='step')
h3 = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t3, density=True, histtype='step')

In [ ]:
np.max(np.array(hit_llh_ps)[:-1]/norm_ps - h[0]), np.max(np.array(hit_llh_ps2)[:-1]/norm_ps2 - h2[0]), np.max(np.array(hit_llh_ps3)[:-1]/norm_ps3 - h3[0])

In [ ]:
h = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
h2 = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2, density=True, histtype='step')
h3 = plt.hist(hits[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d3, density=True, histtype='step')

In [ ]:
np.max(np.array(hit_llh_p)[:-1]/norm_p - h[0]), np.max(np.array(hit_llh_p2)[:-1]/norm_p2 - h2[0]), np.max(np.array(hit_llh_p3)[:-1]/norm_p3 - h3[0])

chargenets

In [ ]:
#%%time
#for i, e in enumerate(test_events):
#    if i == 0:
#        test_charges_t = [[np.sum(e[1]), np.sum(e[1] > 0)]]
#        test_charges_d = np.append(detector, e[1].reshape(-1,1), axis=1)
#    else:
#        test_charges_t = np.concatenate([test_charges_t, [[np.sum(e[1]), np.sum(e[1] > 0)]]])
#        test_charges_d = np.concatenate([test_charges_d, np.append(detector, e[1].reshape(-1,1), axis=1)])

In [ ]:
%%time
charges_t = NNs.get_charge_data(events)[0]
charges_d = NNs.get_dom_data(events, detector)[0]

In [ ]:
exp_bins, exp_bins_fine = np.linspace(0,12,13), np.linspace(15,150,136)

r_t, r_d, N_exp, dom_c_llh = get_charge_data(np.array([3., 1., 0, 0, 0, np.arccos(1), 5., 0.8]),
                                             exp_bins, exp_bins_fine)
r_t2, r_d2, N_exp2, dom_c_llh2 = get_charge_data(np.array([-1., 2., 0, 0, 0, np.arccos(1), 2., 1]),
                                                 exp_bins, exp_bins_fine)
r_t3, r_d3, N_exp3, dom_c_llh3 = get_charge_data(np.array([-7., 1., 0, 0, 0, np.arccos(0), 5., 0.8]),
                                                 exp_bins, exp_bins_fine)

In [ ]:
plt.figure(figsize=(14,7*0.618))

plt.subplot(121)
bins = np.linspace(15,149,68)
#plt.hist(test_charges_t[:, 0], bins, label='Pulses example event', density=True, histtype='step')
plt.hist(charges_t[:, 0], bins, label='p(x)', density=True, histtype='step', color='black')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp)), 
         c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_t2, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp2)), 
         c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_t3, density=True, histtype='step')
plt.plot(exp_bins_fine+np.diff(exp_bins_fine)[0]/2, stats.poisson.pmf(exp_bins_fine, mu=np.sum(N_exp3)), 
         c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.title('All-sensor')
plt.legend()
plt.xlabel(r'$x$ = Charge (total detector)')
plt.ylabel('PDF')


plt.subplot(122)
bins = np.linspace(0,12,13)
#plt.hist(test_charges_d[:, 3], bins, label='Pulses example event', density=True, histtype='step')
plt.hist(charges_d[:, 3], bins, label='p(x)', density=True, histtype='step', color='black')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh/len(N_exp), c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh2/len(N_exp2), c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')

plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3, density=True, histtype='step')
plt.plot(exp_bins+np.diff(exp_bins)[0]/2, dom_c_llh3/len(N_exp3), c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')

plt.title('Per-sensor')
plt.legend()
plt.yscale('log')
plt.ylim(1e-4, 2)
plt.xlabel(r'$x$ = Charge (sensor)')

#plt.savefig('images/spherical/NNtest_spherical_rweight_charge.pdf', bbox_inches='tight')

In [ ]:
bins = np.linspace(15,149,68)
h = plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t, density=True, histtype='step')
h2 = plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t2, density=True, histtype='step')
h3 = plt.hist(charges_t[:, 0], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_t3, density=True, histtype='step')

In [ ]:
np.max(stats.poisson.pmf((bins[1:]+bins[:-1])/2, mu=np.sum(N_exp)) - h[0]),np.max(stats.poisson.pmf((bins[1:]+bins[:-1])/2, mu=np.sum(N_exp2)) - h2[0]), np.max(stats.poisson.pmf((bins[1:]+bins[:-1])/2, mu=np.sum(N_exp3)) - h3[0])

In [ ]:
bins = np.linspace(0,12,13)
h = plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d, density=True, histtype='step')
h2 = plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d2, density=True, histtype='step')
h3 = plt.hist(charges_d[:, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d3, density=True, histtype='step')

In [ ]:
np.max(dom_c_llh[:-1]/len(N_exp) - h[0]), np.max(dom_c_llh2[:-1]/len(N_exp2) - h2[0]), np.max(dom_c_llh3[:-1]/len(N_exp3) - h3[0])

In [ ]:
bins = np.linspace(0,13,14)
plt.figure(figsize=(16,12))
plt.subplots_adjust(wspace=0.25, hspace=0.3)

for i, n in enumerate([0,20,40,45,50,70,100,120,150]):
    plt.subplot(3,3,i+1)
    idx = len(detector)*np.array(range(len(events)))+n
    #plt.hist(test_charges_d[idx, 3], bins, label='Pulses example event', density=True, histtype='step')
    plt.hist(charges_d[idx, 3], bins, label='p(x)', density=True, histtype='step', color='black')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{1})$', weights=r_d[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp[n]), 
             c='tab:blue', linestyle='--', label=r'$p(x|\theta_{1})$')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{2})$', weights=r_d2[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp2[n]), 
             c='tab:orange', linestyle='--', label=r'$p(x|\theta_{2})$')
    
    plt.hist(charges_d[idx, 3], bins, label=r'$p(x) \hat{r}(x,\theta_{3})$', weights=r_d3[idx], density=True, histtype='step')
    plt.plot(exp_bins+np.diff(exp_bins)[0]/2, stats.poisson.pmf(exp_bins, mu=N_exp3[n]), 
             c='tab:green', linestyle='--', label=r'$p(x|\theta_{3})$')
    
    plt.title('DOM '+str(n))
    if i == 2: plt.legend()
    plt.yscale('log')
    if i > 5: plt.xlabel(r'$x$ = Charge (sensor)')
    if i%3 == 0: plt.ylabel('PDF')
    plt.ylim(1e-6, 1e0)
#plt.savefig('images/spherical/NNtest_spherical_rweight_charge_d.png', bbox_inches='tight')

# Reco

In [ ]:
from spherical_opt import spherical_opt
from multiprocessing import Pool, Process

In [ ]:
bounds = np.array([[-9.5,9.5], [-9.5,9.5], [-9.5,9.5], [-200,200], [0,2*np.pi], [0,np.pi], [1,12], [0,1]])
def in_bound(X, radius=9.5):
    r = np.sqrt(np.sum(X[:3]**2))
    r_end = np.sqrt(np.sum(toy_experiment.endpoint(*X)**2))
    return (r<=radius) & (r_end<=radius) & (np.alltrue(np.logical_and(bounds[:,0] <= X, X <= bounds[:,1]), axis=-1))

def init_points(hits, n_live_points, bound=bounds, seed=[None]):
    if seed[0] == None:
        avg = np.average(hits[:, :4], axis=0)
        low_lims = np.concatenate([avg-np.array([3,3,3,40]), np.array([0,0,1,0])])
        hig_lims = np.concatenate([avg+np.array([3,3,3,0]), np.array([2*np.pi,np.pi,12,1])])
    else:
        low_lims = seed - np.array([1, 1, 1, 5, 0.5, 0.3, 3, 3])
        hig_lims = seed + np.array([1, 1, 1, 5, 0.5, 0.3, 3, 3])
    
    uniforms = np.random.uniform(size=(n_live_points, 8))
    initial_points = low_lims + uniforms * (hig_lims - low_lims)
    initial_points = np.clip(initial_points, bounds[:, 0], bounds[:, 1])
    return initial_points

In [ ]:
events, meta = toy_experiment.generate_events(n=10_000, gamma=0, gen_volume="sphere", e_lim=(3,10), inelast_lim=(0,1),
                                              t_width=0, radius=9.5, contained=True, min_hits=3)
truths = NNs.make_truth_array(events)

### Analytic

In [ ]:
def LLH_ana(X, hits, n_obs, form='total', fix=[None], bounds=bounds):
    if fix[0] != None:
        X = np.insert(X, fix[0], fix[1])
        
    if not in_bound(X):
        return 1e9
    
    segments = toy_experiment.model(*X)
    if form == 'dom':
        h_term = toy_experiment.nllh_p_term_dom(segments, hits)
        c_term = toy_experiment.nllh_N_term_dom(segments, n_obs)
    elif form == 'total':
        h_term = toy_experiment.nllh_p_term_tot(segments, hits)
        c_term = toy_experiment.nllh_N_term_tot(segments, n_obs)
    else:
        raise NameError("Formulation must be one of ['total', 'dom'], not "+form)
    
    return c_term + h_term

def fit_event_ana(event):
    hits = np.stack([event.photons[var].to_numpy() for var in ['x', 'y', 'z', 't', 'sensor_id']], axis=1)
    n_obs = event.n_obs.to_numpy() 
    #truth = event?
    
    def eval_LLH(params):
        if params.ndim == 1:
            return LLH_ana(params, hits, n_obs)
        else:
            llhs = []
            for p in params:
                llhs.append(LLH_ana(p, hits, n_obs))
            return np.array(llhs)

    # seeding
    initial_points = init_points(hits, 97) #, seed=truth
    
    start = time.time()
    # free fit
    fit_res = spherical_opt.spherical_opt(
        func=eval_LLH,
        method="CRS2",
        initial_points=initial_points,
        rand=np.random.default_rng(42),
        spherical_indices=[[4,5]],
        batch_size=12,
    )
    fit_res['x'] = np.append(fit_res['x'], time.time() - start)

    return list(fit_res['x'])

In [ ]:
%%time
with Pool(10) as p:
    outs = p.map(fit_event_ana, events)
recos_ana = np.array(outs)

In [ ]:
diff_ana = recos_ana[:, :8] - truths
diff_ana[:, 6] = np.log10(recos_ana[:, 6] / truths[:, 6])
#np.save('recos/spherical/diff_ana', diff_ana)
#np.save('recos/spherical/time_ana', recos_ana[:,8])

### NNs

In [ ]:
import math

from functools import partial
from freedom.llh_service.llh_service import LLHService
from freedom.llh_service.llh_client import LLHClient
from freedom.reco import crs_reco

In [ ]:
NE, Set = 100000, 3

In [ ]:
loc = 'networks/spherical/'
service_conf = {
        "poll_timeout": 1,
        "flush_period": 1,
        "n_hypo_params": 8,
        "n_hit_features": 6,
        "n_evt_features": 2, #len(detector)*4, #
        "batch_size" : {
          "n_hypos": 200,
          "n_observations": 6000, 
        },
        "send_hwm": 10000,
        "recv_hwm": 10000,
        "hitnet_file": loc+'%s/spherical_toy_hitnet_total_%s_set%s.h5'%(NE, NE, Set),
        "chargenet_file": loc+'%s/spherical_toy_chargenet_total_%s_set%s.h5'%(NE, NE, Set),
        #"hitnet_file": loc+'spherical_toy_hitnet_dom.h5',
        #"domnet_file": loc+'spherical_toy_chargenet_dom.h5',
        #"ndoms": len(detector),
        "toy": True
}

In [ ]:
n_gpus = 4

base_req = "ipc:///tmp/recotestreq"
base_ctrl = "ipc:///tmp/recotestctrl"

req_addrs = []
ctrl_addrs = []
for i in range(n_gpus):
    req_addrs.append(f'{base_req}{i}')
    ctrl_addrs.append(f'{base_ctrl}{i}')
    
procs = []
for i in range(n_gpus):
    proc = Process(target=crs_reco.start_service, args=(service_conf, ctrl_addrs[i], req_addrs[i], i))
    proc.start()
    procs.append(proc)

In [ ]:
def fit_events_nn(events, index, Truths, ctrl_addrs):
    outputs = []

    client = LLHClient(ctrl_addr=ctrl_addrs[index], conf_timeout=60000)
    def Eval_llh(params, hits, n_obs, fix=[None]):
        if fix[0] != None:
            params = np.insert(params, fix[0], fix[1])
            
        if not in_bound(params):
            return 1e9
        
        c_data = [np.sum(n_obs), np.sum(n_obs > 0)] #total
        #c_data = np.hstack([detector, n_obs[:, np.newaxis]]) #dom
        return client.eval_llh(hits, c_data, params)

    for j, event in enumerate(events):
        hits = np.stack([event.photons[var].to_numpy() for var in ['x', 'y', 'z', 't', 'q', 'sensor_id']], axis=1)
        n_obs = event.n_obs.to_numpy()
        
        def eval_LLH(params):
            if params.ndim == 1:
                return Eval_llh(params, hits, n_obs)
            else:
                o = []
                for p in params:
                    o.append(Eval_llh(p, hits, n_obs))
                return np.array(o)

        # seeding
        initial_points = init_points(hits, 97) #, seed=Truths[j]
        
        start = time.time()
        #free fit
        fit_res = spherical_opt.spherical_opt(
            func=eval_LLH,
            method="CRS2",
            initial_points=initial_points,
            rand=np.random.default_rng(42),
            spherical_indices=[[4,5]],
            batch_size=12,
        )
        outputs.append(np.append(fit_res['x'], time.time() - start)) #
        
    return outputs

In [ ]:
events_to_process = len(events)
pool_size = 200
evts_per_proc = int(math.ceil(events_to_process/pool_size))
evt_splits = [events[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
true_splits = [truths[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
print(sum(len(l) for l in evt_splits))

gpu_inds = np.arange(pool_size) % n_gpus

fit_events_partial = partial(
        fit_events_nn,
        ctrl_addrs=ctrl_addrs
)

In [ ]:
%%time
# reconstruct with a worker pool; one LLH client per worker
with Pool(pool_size) as p:
    outs = p.starmap(fit_events_partial, zip(evt_splits, gpu_inds, true_splits))

all_outs = sum((out for out in outs), [])
all_outs = np.array(all_outs).reshape((events_to_process, 9))
recos_nn = np.array(all_outs)

In [ ]:
diff_nn = recos_nn[:, :8] - truths
diff_nn[:, 6] = np.log10(recos_nn[:, 6] / truths[:, 6])
np.save('recos/spherical/%s/diff_nn_total_%s_set%s'%(NE, NE, Set), diff_nn) #perDOM
#np.save('recos/spherical/time_total', recos_nn[:,8]) #perDOM

In [ ]:
# kill all the services
import zmq
for proc, ctrl_addr in zip(procs, ctrl_addrs): 
    with zmq.Context.instance().socket(zmq.REQ) as ctrl_sock:
        ctrl_sock.connect(ctrl_addr)
        ctrl_sock.send_string("die")
        proc.join()

### plot

In [ ]:
diff_ana = np.load('recos/spherical/diff_ana.npy')
diff_ana2 = np.load('recos/spherical/diff_ana2.npy')
#diff_nn_t = np.load('recos/spherical/diff_nn_total.npy')
#diff_nn_d = np.load('recos/spherical/diff_nn_perDOM.npy')

NEs = np.array([50000, 100000, 200000, 500000, 1000000, 1500000])

for s in range(5):
    for NE in NEs:
        exec("diff_nn_%s_set%s = np.load('recos/spherical/%s/diff_nn_total_%s_set%s.npy')"%(NE, s, NE, NE, s))

In [ ]:
lims = [[-2,2],[-2,2],[-2,2],[-5,5],[-2*np.pi,2*np.pi],[-np.pi,np.pi],[-0.2,0.2],[-0.7,0.7]]

fig = plt.figure(figsize=(25, 17))

for i in range(8):
    bins = np.linspace(lims[i][0], lims[i][1], 50)
    
    plt.subplot(3,3,i+1)
    plt.hist(diff_ana[:, i], bins, label='Analytic', alpha=0.5, histtype='step', linewidth=3, density=True)
    #plt.hist(diff_nn_t[:, i], bins, label='NN total', alpha=0.5, histtype='step', linewidth=3, density=True)
    #plt.hist(diff_nn_d[:, i], bins, label='NN dom', alpha=0.5, histtype='step', linewidth=3, density=True)
    plt.hist(diff_nn_1500000_set0[:, i], bins, label='NN', alpha=0.5, histtype='step', linewidth=3, density=True)
    #plt.hist(recos_ana[:, i], 50, alpha=0.5, label='Analytic')
    #plt.hist(recos_nn_t[:, i], 50, alpha=0.5, label='NN (total)')
    #plt.hist(recos_nn_d[:, i], 50, alpha=0.5, label='NN (dom)')
    if i == 2: plt.legend()
    if i == 1: plt.title('Spherical Detector, Reco-True')
    plt.xlabel(par_names[i])
    
#plt.savefig('images/spherical/spherical_reco', bbox_inches='tight')

ks results

In [ ]:
ks_ana2 = []
for i in range(8):
    ks_ana2.append(stats.ks_2samp(diff_ana[:,i], diff_ana2[:,i])[0])

kss = np.zeros((5,len(NEs),8))  
for s in range(5):
    for j, NE in enumerate(NEs):
        exec('diff = diff_nn_'+str(NE)+'_set'+str(s))
        for i in range(8):
            kss[s][j][i] = stats.ks_2samp(diff_ana[:,i], diff[:,i])[0]

In [ ]:
fig = plt.figure(figsize=(7, 8*0.618))
ax1 = fig.add_subplot(111)
#ax2 = ax1.twiny()

ax1.errorbar(NEs, np.mean(np.mean(kss, axis=2), axis=0), np.std(np.mean(kss, axis=2), axis=0), 
             label='Average all sets', color='black')
for i, a in enumerate(np.mean(kss, axis=2)):
    ax1.scatter(NEs, a, label='Average set '+str(i)) #

ax1.axhline(np.mean(ks_ana2), color='black', linestyle='--', label=r'KS$_{rereco}$')

#ax2.set_xlabel('Data size (GB)')
#ax2.scatter(data_sizes[::2]*1e-9, np.mean(kss[0], axis=1), marker='')
#ax2.set_xscale('log')

ax1.legend(prop={'size':15}, loc='upper right')
ax1.set_xscale('log')
ax1.set_xlabel('#Events in training set')
ax1.set_ylabel('KS value')
plt.ylim(0, 0.3)

#plt.savefig('images/spherical/spherical_reco_ks_avg_sets.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,7))
for i in range(8):
    plt.scatter(NEs, kss[0][:,i], alpha=0.5, label=par_names[i])
plt.scatter(NEs, np.mean(kss[0], axis=1), label='Average', color='black')
plt.axhline(np.mean(ks_ana2), color='black', linestyle='--', label=r'KS$_{rereco}$')

plt.legend(prop={'size':15})
plt.xscale('log')
plt.xlabel('#Events in training set')
plt.ylabel('KS value')

#plt.savefig('images/spherical/spherical_reco_ks.png', bbox_inches='tight') #_avg

timing

In [ ]:
time_nn_t = recos_nn[:,8] #np.load('recos/spherical/time_total.npy')
time_nn_d = np.load('recos/spherical/time_perDOM.npy')

PD = dm.PointData({'time_t': time_nn_t[:1000],
                   'time_d': time_nn_d,
                  })

In [ ]:
bins = np.linspace(0,100,1000)
PD.kde(time_t=bins).density.plot(label='All-sensor (mean=%.1f s)'%(np.mean(PD.time_t)))
PD.kde(time_d=bins).density.plot(label='Per-sensor (mean=%.1f s)'%(np.mean(PD.time_d)))

plt.legend()
plt.xlabel('Reconstruction time (s)')
plt.ylabel('Density')

#plt.savefig('images/spherical/timing_spherical', bbox_inches='tight')

In [ ]:
n_hits = NNs.get_charge_data(events)[0][:,0]
mean_hits = np.mean(n_hits)

(mean_hits+len(detector))/(mean_hits+1), np.mean(time_nn_d)/np.mean(time_nn_t)

In [ ]:
mean_hits